**PENJELASAN**


Program ini dibuat untuk membantu menentukan kos terbaik dengan
menggunakan Sistem Fuzzy. Penilaian kos dilakukan berdasarkan
dua kriteria utama, yaitu fasilitas yang dimiliki kos dan harga
sewa kos.

Pada program ini, proses sistem fuzzy dilakukan secara lengkap,
mulai dari fuzzifikasi, inferensi, hingga defuzzifikasi. Seluruh
proses tersebut dibuat secara manual tanpa menggunakan library
Fuzzy Systems.

Data kos dibaca dari file Excel, kemudian setiap kos diproses
untuk menghasilkan nilai skor kelayakan. Hasil akhir dari program
ini adalah daftar 5 kos terbaik yang memiliki skor kelayakan
tertinggi.






**IMPORT & BACA DATA DARI FILE**

In [13]:
# Import library untuk membaca file Excel
from openpyxl import load_workbook

# Membuka file Excel dataset kos
wb = load_workbook("dataset_kos.xlsx")
ws = wb.active

# Menyimpan data ke dalam list of dictionary
data = []
for row in ws.iter_rows(min_row=2, values_only=True):
    data.append({
        "id_kos": row[0],
        "fasilitas": row[1],
        "harga": row[2]
    })

# Cek jumlah data
len(data)

50

**FUZZIFIKASI FASILITAS**

In [14]:
# Fuzzifikasi fasilitas kos

def fasilitas_rendah(x):
    if x <= 50:
        return 1
    elif x <= 70:
        return (70 - x) / 20
    else:
        return 0

def fasilitas_sedang(x):
    if 60 <= x <= 80:
        return 1 - abs(x - 70) / 10
    else:
        return 0

def fasilitas_tinggi(x):
    if x >= 90:
        return 1
    elif x >= 70:
        return (x - 70) / 20
    else:
        return 0

In [15]:
# Contoh uji fuzzifikasi fasilitas
x = 75

print("Fasilitas Rendah :", fasilitas_rendah(x))
print("Fasilitas Sedang :", fasilitas_sedang(x))
print("Fasilitas Tinggi :", fasilitas_tinggi(x))


Fasilitas Rendah : 0
Fasilitas Sedang : 0.5
Fasilitas Tinggi : 0.25


**FUZZIFIKASI HARGA**

In [16]:
# Fuzzifikasi harga kos (dalam Rupiah)

def harga_murah(x):
    if x <= 700000:
        return 1
    elif x <= 1000000:
        return (1000000 - x) / 300000
    else:
        return 0

def harga_sedang(x):
    if 800000 <= x <= 1300000:
        return 1 - abs(x - 1050000) / 250000
    else:
        return 0

def harga_mahal(x):
    if x >= 1500000:
        return 1
    elif x >= 1000000:
        return (x - 1000000) / 500000
    else:
        return 0

In [17]:
#fuzzifikasi harga kos
harga = 900000

print("Harga Murah  :", harga_murah(harga))
print("Harga Sedang :", harga_sedang(harga))
print("Harga Mahal  :", harga_mahal(harga))


Harga Murah  : 0.3333333333333333
Harga Sedang : 0.4
Harga Mahal  : 0


**PROSES FUZZIFIKASI DATA**

In [18]:
# FUZZIFIKASI SETIAP DATA KOS
for d in data:
    f = d["fasilitas"]
    h = d["harga"]

    d["fuzzy"] = {
        "fasilitas_rendah": fasilitas_rendah(f),
        "fasilitas_sedang": fasilitas_sedang(f),
        "fasilitas_tinggi": fasilitas_tinggi(f),

        "harga_murah": harga_murah(h),
        "harga_sedang": harga_sedang(h),
        "harga_mahal": harga_mahal(h)
    }


**INFERENSI**

In [19]:
# INFERENSI FUZZY


def inferensi(f):
    rules = []

    # Rule 1
    rules.append(min(f["fasilitas_tinggi"], f["harga_murah"]) * 1.0)

    # Rule 2
    rules.append(min(f["fasilitas_tinggi"], f["harga_sedang"]) * 0.9)

    # Rule 3
    rules.append(min(f["fasilitas_sedang"], f["harga_murah"]) * 0.8)

    # Rule 4
    rules.append(min(f["fasilitas_sedang"], f["harga_sedang"]) * 0.7)

    # Rule 5
    rules.append(min(f["fasilitas_rendah"], f["harga_murah"]) * 0.6)

    # Rule 6
    rules.append(f["harga_mahal"] * 0.4)

    # Rule 7
    rules.append(min(f["fasilitas_rendah"], f["harga_mahal"]) * 0.2)


    # Mengambil nilai maksimum sebagai hasil inferensi (operator OR).
    return max(rules)


**DEFUZZIFIKASI**

In [20]:
# DEFUZZIFIKASI

# Mengubah hasil inferensi fuzzy menjadi satu nilai skor kelayakan untuk setiap kos.
for d in data:
    d["skor_kelayakan"] = inferensi(d["fuzzy"])

**MENYORTING & MENGAMBIL 5 KOS TERBAIK**


In [21]:
# Mengurutkan data berdasarkan skor kelayakan tertinggi
data_sorted = sorted(data, key=lambda x: x["skor_kelayakan"], reverse=True)

# Mengambil 5 kos terbaik
top5 = data_sorted[:5]

In [22]:
data_sorted = sorted(data, key=lambda x: x["skor_kelayakan"], reverse=True)

print("===== 5 KOS TERBAIK BERDASARKAN SISTEM FUZZY =====\n")

for i, d in enumerate(data_sorted[:5], start=1):
    print(f"Peringkat {i}")
    print(f"ID Kos         : {d['id_kos']}")
    print(f"Fasilitas     : {d['fasilitas']}")
    print(f"Harga         : {d['harga']}")
    print(f"Skor Kelayakan: {round(d['skor_kelayakan'], 3)}")
    print("-" * 40)


===== 5 KOS TERBAIK BERDASARKAN SISTEM FUZZY =====

Peringkat 1
ID Kos         : K15
Fasilitas     : 83
Harga         : 1150000
Skor Kelayakan: 0.54
----------------------------------------
Peringkat 2
ID Kos         : K35
Fasilitas     : 82
Harga         : 1160000
Skor Kelayakan: 0.504
----------------------------------------
Peringkat 3
ID Kos         : K19
Fasilitas     : 81
Harga         : 1080000
Skor Kelayakan: 0.495
----------------------------------------
Peringkat 4
ID Kos         : K17
Fasilitas     : 66
Harga         : 820000
Skor Kelayakan: 0.48
----------------------------------------
Peringkat 5
ID Kos         : K33
Fasilitas     : 67
Harga         : 830000
Skor Kelayakan: 0.453
----------------------------------------


**MENYIMPAN OUTPUT KE DALAM FILE**

In [23]:
# Menyimpan hasil 5 kos terbaik ke file teks
with open("hasil_5_kos_terbaik.txt", "w") as file:
    file.write("===== 5 KOS TERBAIK BERDASARKAN SISTEM FUZZY =====\n\n")

    for i, d in enumerate(data_sorted[:5], start=1):
        file.write(f"Peringkat {i}\n")
        file.write(f"ID Kos        : {d['id_kos']}\n")
        file.write(f"Fasilitas     : {d['fasilitas']}\n")
        file.write(f"Harga         : {d['harga']}\n")
        file.write(f"Skor Kelayakan: {round(d['skor_kelayakan'], 3)}\n")
        file.write("-" * 40 + "\n")